In [4]:
import numpy as np
import pandas as pd

In [19]:
metadata=pd.read_csv('movies_metadata2.csv',low_memory=False)
metadata.shape
metadata.head(4)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,30-10-95,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,15-12-95,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,22-12-95,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,22-12-95,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34


In [6]:
c=metadata['vote_average'].mean()
print(c)

6.067978357107713


In [7]:
m=metadata['vote_count'].quantile(0.90)
print(m)

853.5999999999999


In [8]:
q_movies=metadata.copy().loc[metadata['vote_count']>=m]
q_movies.shape

(204, 24)

In [9]:
def rating(x,m=m,c=c):
    v=x['vote_count']
    R=x['vote_average']
    return (v/v+m*R)+(m/m+v*c)

In [10]:
q_movies['score']=q_movies.apply(rating,axis=1)

In [11]:
q_movies=q_movies.sort_values('score',ascending=False)

In [12]:
q_movies[['title','vote_count','vote_average','score']].head(5)

,title,vote_count,vote_average,score
292,Pulp Fiction,8670,8.3,59696.252356
314,The Shawshank Redemption,8358,8.5,57973.763109
351,Forrest Gump,8147,8.2,56437.339675
1639,Titanic,7770,7.5,53552.191835
256,Star Wars,6778,8.1,48044.917304


In [13]:
 metadata['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(stop_words='english')
metadata['overview']=metadata['overview'].fillna('')
matrix=tfidf.fit_transform(metadata['overview'])


In [15]:
matrix.shape

(2033, 14018)

In [17]:
from sklearn.metrics.pairwise import linear_kernel
cosine=linear_kernel(matrix,matrix)

In [29]:
cosine.shape

(2033, 2033)

In [18]:
indices=pd.Series(metadata.index,index=metadata['title']).drop_duplicates()

In [30]:
indices.head()

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64

In [27]:
def get_recommendations(title, cosine=cosine):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:20]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return metadata['title'].iloc[movie_indices]

In [28]:
get_recommendations('The Shawshank Redemption')

1231                      Cool Hand Luke
1564                   Alive and Kicking
499                            No Escape
454                          The Getaway
1686                   A Further Gesture
1204                     La Femme Nikita
1441                         Hollow Reed
302                    Three Colors: Red
812                       Chain Reaction
615                        Condition Red
683                    Feeling Minnesota
1915                         The Rapture
1115                            Raw Deal
0                              Toy Story
1658                           Firestorm
666     Some Folks Call It a Sling Blade
1392                          Gridlock'd
1070                     American Strays
1273                              Alien³
Name: title, dtype: object